Цель - написать программу, подбирающую подходящие породы собак по запросам пользователя. Для этого необходимо собрать данные с сайта о породах, создать датафрейм и с его помощью фильтровать породы по совпадениям с вводом пользователя. Для этого импортирую необходимые библиотеки.

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import re
from google.colab import files
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt_tab')
from nltk.probability import FreqDist
from wordcloud import WordCloud
import matplotlib.pylab as plt
!pip install pymorphy3
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()
from collections import Counter
!pip install pytelegrambotapi
import telebot
from telebot import types


Парсинг главной страницы, на которой есть список всех пород.

In [36]:
url = 'https://lapkins.ru/dog/'
page1 = requests.get(url)
time.sleep(3)
soup = BeautifulSoup(page1.text, features="html.parser")

Создание списка Порода+ссылка на страницу породы.

In [ ]:
base_url = "https://lapkins.ru"
poroda_elements = soup.find_all('a', class_='poroda-element')
porody = [] #список всех пород на главной странице
for poroda in poroda_elements:
    name = poroda.find('span').get_text() # название породы
    href = poroda['href']
    full_url = base_url + href #ссылка на каждую породу на странице
    porody.append((name, full_url))
for name, url in porody:
    #print(f"Название породы: {name}, Ссылка: {url}")

Парсинг каждой породы с главной страницы. Занимает около 20 минут. На этом шаге пришлось из jupyter'а перейти в Colab, потому что jupyter не справлялся (постоянно терял связь с сервером и выдавал ошибку).

In [ ]:
all_soups = []

for dogs in range(len(porody)):
    url = porody[dogs][1]  #ссылки на страницы породы
    response = requests.get(url, timeout=10)
    time.sleep(5)

    soup2 = BeautifulSoup(response.text, 'html.parser')
    all_soups.append(soup2)



Просмотр "супа" по породам.

In [ ]:
#print(all_soups[0])

Составление датафрейма по всем породам.
Сначала извлекаю Рост, так как он находится в шапке страницы, внутри отдельного тега. У всех пород он указан в разном формате (одним числом, двумя числами через разные тире/дефис, знак погрешности), у некоторых вообще не указан.
Затем извлекаю остальные нужные характеристики. Они находятся в одном блоке.
Потом добавляю два фрагмента из описания породы.
Тут же добавляю возможность скачивания файла в корректном формате Excel, чтобы было удобно проверять данные.

In [ ]:
def dog_height(soup):   #рост
    info_list = soup.find('ul', class_='info') #тег, содержащий нужную информацию о породе
    if not info_list:
        return "Не указано"

    height = "Не указано"

    li_elements = info_list.find_all('li') #отдельные элементы внутри тега
    for li in li_elements:
        text = li.text.strip()

        if 'рост' in text.lower():
            try:
                height_match = re.search(r'(\d{1,3}\s?±?\s?\d{0,3}\s?см)|(\d{1,3}\s?–\s?\d{1,3}\s?см)', text, re.IGNORECASE) #такое регулярное выражение захватывает максимальное количество корректных данных
                if height_match:
                    height = height_match.group(1).strip() if height_match.group(1) else height_match.group(2).strip()
            except IndexError:
                pass

    return height


def character(soup):
    characteristics = {
        "Агрессивность": "",
        "Активность": "",
        "Дрессировка": "",
        "Линька": "",
        "Дружелюбность": "",
        "Отношение к одиночеству": "",
        "Интеллект": "",
        "Шум": "",
        "Охранные качества": ""
    }


    for char in characteristics.keys():
        char_element = soup.select_one(f"div.s-title:contains('{char}')")
        if char_element:
            rating = char_element.find_next('div', class_='s-text')
            if rating:
                characteristics[char] = rating.text.strip()

    return characteristics


def get_description(soup): #частично извлекаю описание породы, чтобы потом поработать с текстом
    text1 = soup.find(id="gl3")  #раздел "Характер"
    text2 = soup.find(id="gl4")  #раздел "Воспитание и дрессировка"

    if not text1 or not text2:
        return "Не указано"

    texts = []


    for x in text1.find_next_siblings():
        if x.name == 'h2' and x.get('id') == "gl4":
            break
        if x.name == 'p':
            texts.append(x.get_text())


    for x in text2.find_next_siblings():
        if x.name == 'h2':
            break
        if x.name == 'p':
            texts.append(x.get_text())


    description = '\n'.join(texts) #объединение двух разделов для удобства работы
    return description


breed_data = []
for i, soup in enumerate(all_soups):  # данные по каждой породе
    breed_name = porody[i][0]  # название породы
    rost = dog_height(soup)  # рост
    characteristics = character(soup)  # характеристики
    description = get_description(soup)  # получаем описание для каждой породы

    breed_data.append({
        "Порода": breed_name,
        "Рост": rost,
        "Описание": description,
        **characteristics
    })

df = pd.DataFrame(breed_data)

file_name = "/content/breeds_characteristics.csv"
df.to_csv(file_name, index=False, encoding='utf-8-sig', sep=';')  # сохранение файла в корректном для excel формате (для самопроверки)

files.download(file_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Обработка датафрейма. При помощи столбца "Рост" создаю ещё один столбец "Размер". Теперь вместо разных сантиметров в таблице есть столбец с тремя вариантами "Маленькая", "Средняя" и "Крупная". Это будет удобнее для пользователя. Также в графах с характеристиками пород информация записана в формате "Активность: Умеренная (Рейтинг 3/5)". Убираю лишний текст, оставляю только одну нужную цифру.

In [ ]:
def extract_height(height_str):
    if '–' in height_str or '-' in height_str or '±' in height_str: #для строк с диапазоном оставляю только первое число во избежание проблем в пограничных случаях
        numbers = re.findall(r'\d+', height_str)
        if len(numbers) >= 1:
            return int(numbers[0])

    numbers = re.findall(r'\d+', height_str)
    if len(numbers) == 1:
        return int(numbers[0])

    return None

def classify_size(height_value):
    if height_value is None:
        return 'Не указано'

    if height_value <= 40:
      return 'Маленькая'
    elif 41 <= height_value <= 60:
      return 'Средняя'
    elif height_value >= 61:
      return 'Крупная'
    return 'Не указано'

df['Размер'] = df['Рост'].apply(extract_height).apply(classify_size)

def extract_rating(text):  # оставляю в строке только одну нужную цифру
    if isinstance(text, str) and text:
        match = re.findall(r'Рейтинг\s*(\d)\s*/\d', text)
        if match:
            return match[0]
    return ''

columns_to_process = ['Агрессивность', 'Активность', 'Линька', 'Дрессировка', 'Дружелюбность',
                      'Отношение к одиночеству', 'Интеллект', 'Шум', 'Охранные качества']

for column in columns_to_process:
    df[column] = df[column].apply(extract_rating)


small_breeds = df[df['Размер'] == 'Маленькая']['Порода'].tolist()
medium_breeds = df[df['Размер'] == 'Средняя']['Порода'].tolist()
large_breeds = df[df['Размер'] == 'Крупная']['Порода'].tolist()

file_name = "/content/breeds_characteristics.csv"
df.to_csv(file_name, index=False, encoding='utf-8-sig', sep=';')  # сохранение файла в корректном для excel формате (для самопроверки)

files.download(file_name)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Когда информация в датафрейме полностью подготовлена, приступаю к коду взаимодействия с пользователем. Задаю вопрос про размер собаки, добавляя возможность вводить разные ответы.
Далее следуют вопросы по характеристикам породы. Результат выводится не только по точному совпадению с выбранным индексом, но и с лучшими условиями (например, если пользователь указал, что готов к собаке с уровнем линьки 3/5, программа предложит ему и породы с индексом 1 и 2).
Результат выдаёт пользователю название подходящих пород, а также ссылки на страницы с полным описанием и фотографиями.

In [ ]:
def ask_for_size():
    while True:
        size_input = input("Какой размер собаки вам подходит: маленькая (до 40см в холке), средняя (41-60см в холке), крупная (выше 60 см в холке)? Вы можете написать два варинанта или 'неважно'): ").strip().lower()

        if size_input in ["неважно"]:
            return None

        sizes = [s.strip().capitalize() for s in size_input.replace(",", " ").split()] #обработка ввода пользователя в вопросе про размер

        valid_sizes = {"Маленькая", "Средняя", "Крупная"}
        if all(s in valid_sizes for s in sizes):
            return sizes
        else:
            print("Пожалуйста, выберите один или несколько из следующих вариантов: Маленькая, Средняя, Крупная.")


def filter_by_size(df, sizes):
    if sizes is None:
        return df
    return df[df['Размер'].isin(sizes)]

user_size = ask_for_size() #размер собаки

bot_questions = {
    "Агрессивность": "Теперь Вы должны ответить на несколько вопросов, касающихся характера Вашего будущего питомца. Для начала поговорим о такой черте как агрессивность. Это слово может напугать, однако на деле всё не так страшно. Речь не идёт о злости и желании разорвать на кусочки каждого встречного. Собаки с высоким индексом агрессивности более склонны защищать свои личные границы и ресурсы, в том числе и своего человека. Они бывают недоверчивы к незнакомцам, могут охранять свою миску с едой или сцепиться на улице с другой собакой. Чтобы избежать такого поведения, хозяину необходимо тщательно дрессировать своего питомца. Оцените от 1 до 5 свою готовность заниматься собакой с возможными проявлениями агрессии.",
    "Активность": "Следующий пункт - активность. Если Вы выбираете активную собаку, будьте готовы гулять не меньше двух-трёх часов в день, ездить на природу и играть в активные игры. А некоторым породам даже рекомендуются занятия специальными видами спортами (аджилити, фрисби, пастьба и т.п.). Любителям провести выходной на диване перед телевизором такой питомец не подойдёт. Напишите, к какому уровню активности Вы готовы (от 1 до 5).",
    "Дрессировка": "Перейдём к дрессировке. Абсолютно любую собаку можно научить выполнять базовые команды вроде 'сидеть', 'лежать' и 'ко мне'. Для комфортной жизни этого достаточно. Если же Вы мечтаете иметь питомца, знающего сотню разных команд, лучше хорошенько подумать над этим пунктом, чтобы потом не разочароваться в своём выборе. Оцените свои требования к обучаемости собаки от 1 до 5.",
    "Линька": "Поговорим о шерсти. Готовы ли Вы часто мыть, вычёсывать и водить на груминг своего четвероногого друга? А тщательно перебирать его мех в поиске клещей после прогулки в лесу? А во время линьки проводить ежедневную уборку и находить шерсть даже в своём завтраке? Пожалуйста, оцените свою готовность цифрой от 1 до 5.",
    "Дружелюбность": "Есть собаки, которые любят всех вокруг: чужих взрослых и детей, соседских собак, дворовых кошек и голубей. А есть такие, которых интересуют только члены 'своей стаи'. Они самодостаточны и не будут вилять хвостиком каждому гостю или терпеть приставания чужих детей. Решите, насколько дружелюбную собаку Вы хотели бы иметь (от 1 до 5)",
    "Отношение к одиночеству": "Теперь вопрос о Вашем образе жизни. Если Вы живёте один(одна), много работаете и планируете надолго оставлять собаку дома, лучше не выбирать породу, которая плохо переносит одиночество. Пока Вас не будет дома, такая собака может от тоски начать выть или крушить квартиру. Для комфортного сосуществования необходимо выбирать породу, подходящую под Ваш ритм жизни. Поэтому сейчас Вам нужно оценить, насколько хорошо пёс должен уметь оставиться один дома (как и ранее, от 1 до 5)",
    "Интеллект": "Интеллект собаки. Скорее всего, Вам сразу же захочется поставить в этом пунтке '5', но не спешите. Во-первых, Вы сильно ограничите список рекомендаций, потому что таких пород не так уж много. Во-вторых, очень умные собаки - это не только интересно, но и крайне сложно. Они очень изобретательны - для них проще простого научиться открывать межкомнатные двери или выдвижные ящики на кухне. И если не давать им достаточной интеллектуальной нагрузки, они сами находят себе развлечения (например, поедение плинтусов). Но если Вы сможете направить разум собаки в нужное русло, будете регулярно уделять время дрессировке, а также купите (или сделаете своими руками) развивающие игрушки, то Вы получите потрясающего друга, который каждый день будет удивлять Вас своей сообразительностью. Выбор за Вами. Ваши требования к интеллекту собаки по пятибалльной шкале ",
    "Шум": "Собаки лают. Само собой, любой человек, решившийся завести собаку, осознаёт, что время от времени его питомец может издавать разные громкие звуки. Но и среди собак есть свои 'молчуны', а есть любители пошуметь. Насколько Вы (и Ваши соседи) готовы к шуму? От 1 до 5.",
    "Охранные качества": "Оцените от 1 до 5, насколько Вам важны охранные качества собаки."
}

def ask_user_for_characteristic(characteristic_name):
    while True:
        try:
            print(bot_questions[characteristic_name])
            user_input = int(input())
            if 1 <= user_input <= 5:
                return user_input
            else:
                print("Число должно быть от 1 до 5!")
        except ValueError:
            print("Пожалуйста, введите число от 1 до 5.")


user_inputs = {key: ask_user_for_characteristic(key) for key in bot_questions}

columns = ['Агрессивность', 'Активность', 'Дрессировка', 'Линька', 'Дружелюбность', 'Отношение к одиночеству', 'Интеллект', 'Шум', 'Охранные качества']
for col in columns:
    df[col] = pd.to_numeric(df[col], errors='coerce') #строки датафрейма перевожу в числовой формат

conditions = []
for column, user_value in user_inputs.items():
    if column in ['Агрессивность', 'Активность', 'Линька', 'Шум']:  #для этих характеристик условие - равно или меньше
        conditions.append(df[column] <= user_value)
    else:  #для остальных - равно или больше
        conditions.append(df[column] >= user_value)

filtered_df = df
for condition in conditions:
    filtered_df = filtered_df.loc[condition]


filtered_df = filter_by_size(filtered_df, user_size)

if not filtered_df.empty:
    print("\nПодходящие Вам породы собак:")
    for _, row in filtered_df.iterrows():
        breed_name = row['Порода']
        breed_url = next((url for name, url in porody if name == breed_name), None)
        if breed_url:
            print(f"Порода: {breed_name}, Ссылка: {breed_url}")
else:
    print("\nК сожалению, подходящих пород не найдено. Возможно, Вам следует немного смягчить условия поиска.")

Какой размер собаки вам подходит: маленькая (до 40см в холке), средняя (41-60см в холке), крупная (выше 60 см в холке)? Вы можете написать два варинанта или 'неважно'): неважно
Теперь Вы должны ответить на несколько вопросов, касающихся характера Вашего будущего питомца. Для начала поговорим о такой черте как агрессивность. Это слово может напугать, однако на деле всё не так страшно. Речь не идёт о злости и желании разорвать на кусочки каждого встречного. Собаки с высоким индексом агрессивности более склонны защищать свои личные границы и ресурсы, в том числе и своего человека. Они бывают недоверчивы к незнакомцам, могут охранять свою миску с едой или сцепиться на улице с другой собакой. Чтобы избежать такого поведения, хозяину необходимо тщательно дрессировать своего питомца. Оцените от 1 до 5 свою готовность заниматься собакой с возможными проявлениями агрессии.
4
Следующий пункт - активность. Если Вы выбираете активную собаку, будьте готовы гулять не меньше двух-трёх часов в день, е

Код ниже оставлен для меня. Была идея помимо названия породы и ссылки на сайт выводить пользователю также облако слов, собранное из самых частых прилагательных в описании породы. Но ожидания не совпали с реальностью, и облака слов получились бредовыми (сколько бы слов я не добавляла к списку стоп-слов).
Поскольку эта идея возникла не столько с целью улучшить программу, сколько просто для демонстрации полученных на курсе умений работать с текстом, я не стала мучиться и просто убрала этот блок, оставив кусочек кода себе на будущее.

In [ ]:
#код-артефакт :)

stop_words = stopwords.words('russian')
stop_words = stop_words + ['который', 'каждый', 'готовый', 'самый', 'ранний', 'присущий', 'элементарный', 'японский',
                         'среднеазиатский', 'русский', 'полный', 'плотный', 'настоящий', 'общий', 'должный', 'чреватый', 'новый',
                         'домашний,' 'определенный', 'общий', 'ваш', 'свой', 'весь', 'постоянный', 'австралийский', 'американский', 'китайский', 'йоркширский', 'хороший',
                         'швейцарский', 'регулярный', 'английский', 'любой', 'собственный', 'главный', 'прямой', 'основной', 'остальной', 'бернский', 'гончий',
                         'русский', 'первый', 'базовый', 'задний', 'нижний', 'некоторый', 'правильный']



def preprocess_text(text_description):
    text_low = text_description.lower()
    text_no_punkt = re.sub(r'[^\w\s]', ' ', text_low)
    tokens = nltk.word_tokenize(text_no_punkt, language='russian')
    lemmatized_tokens = [morph.parse(token)[0].normal_form for token in tokens]
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]

    return filtered_tokens


def extract_adjectives(text_description):
    list_of_adj = []
    filtered_tokens = preprocess_text(text_description)

    for token in filtered_tokens:
        word_parsed = morph.parse(token)[0]
        if word_parsed.tag.POS == 'ADJF' or word_parsed.tag.POS == 'ADJS':
            list_of_adj.append(word_parsed.normal_form)

    return list_of_adj


def generate_wordcloud(adjectives):
    if not adjectives:
        print("Нет прилагательных для создания облака слов.")
        return
    adjective_counts = Counter(adjectives)
    wordcloud = WordCloud(width=800, height=400, background_color='white', max_words=7).generate_from_frequencies(adjective_counts)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()


for i, row in df.iterrows():
    breed_name = row['Порода']
    description = row.get('Описание', None)

    if not description:
        print(f"Нет описания породы: {breed_name}")
        continue

    adjectives = extract_adjectives(description)

    most_common_adjectives = [item[0] for item in Counter(adjectives).most_common(5)]
    print(f" {breed_name}: {most_common_adjectives}")

    generate_wordcloud(most_common_adjectives)


Превращение в телеграм-бот. Бот здоровается с пользователем, спрашивает сначала про размер, потом поочерёдно задаёт вопросы про черты характера. На основании ответов предлагает список подходящих пород со ссылками на их страницы на сайте. Если подходящие породы не найдены, бот предлагает пройти тест заново и смягчить требования.

In [ ]:
my_bot = '7938408611:AAH_qN70aTGzkP9ImDiwzQRr0mc9wzWOUk8'
bot = telebot.TeleBot(my_bot)

df = pd.read_csv('breeds_characteristics (85).csv', sep=';')

bot_questions = {
    "Агрессивность": "Для начала поговорим об агрессивности. Речь не идёт о злости и желании разорвать на кусочки каждого встречного. Собаки с высоким индексом агрессивности более склонны защищать свои личные границы и ресурсы (в том числе и своего человека). Они бывают недоверчивы к незнакомцам, могут охранять свою миску или сцепиться на улице с другой собакой. Для коррекции такого поведения необходима более тщательная дрессировка. Оцените от 1 до 5 свою готовность прорабатывать возможные проявления агрессии.",
    "Активность": "Следующий пункт - активность. Если Вы выбираете активную собаку, будьте готовы гулять не меньше двух-трёх часов в день, ездить на природу и играть в активные игры. А некоторым породам даже рекомендуются занятия специальными видами спортами (аджилити, фрисби, пастьба и т.п.). Любителям провести выходной на диване перед телевизором такой питомец не подойдёт. Выберите, к какому уровню активности Вы готовы.",
    "Дрессировка": "Перейдём к дрессировке. Абсолютно любую собаку можно научить выполнять базовые команды вроде \"сидеть\", \"лежать\" и \"ко мне\". Для комфортной жизни этого достаточно. Если же Вы мечтаете иметь питомца, знающего сотню разных команд, лучше выбрать собаку, легко поддающуюся дрессировке. Оцените свои требования к обучаемости собаки.",
    "Линька": "Поговорим о шерсти. Готовы ли Вы часто мыть, вычёсывать и водить на груминг своего четвероногого друга? А тщательно перебирать его мех в поиске клещей после прогулки в лесу? А во время линьки проводить дома ежедневную уборку? Пожалуйста, оцените свою готовность.",
    "Дружелюбность": "Есть собаки, которые любят всех вокруг: чужих взрослых и детей, соседских собак, дворовых кошек и голубей. А есть такие, которых интересуют только члены \"своей стаи\". Они самодостаточны и не будут вилять хвостиком каждому гостю или терпеть приставания чужих детей. Решите, насколько дружелюбную собаку Вы хотели бы иметь.",
    "Отношение к одиночеству": "Теперь вопрос о Вашем образе жизни. Если Вы живёте один(одна), много работаете и планируете надолго оставлять собаку дома, лучше не выбирать породу, которая плохо переносит одиночество. Пока Вас не будет дома, такая собака может от тоски начать выть или крушить квартиру. Для комфортного сосуществования необходимо выбирать породу, подходящую под Ваш ритм жизни. Поэтому сейчас Вам нужно оценить, насколько хорошо пёс должен уметь оставаться один дома.",
    "Интеллект": "Интеллект собаки. Не спешите сразу выбирать \"5\" в этом пункте. Во-первых, таких пород не так уж много. Во-вторых, очень умные собаки - это зачастую крайне сложно. Они изобретательны - для них проще простого научиться открывать двери или шкафы, а недостаток интеллектуальной нагрузки часто приводит к деструктивному поведению. Зато при регулярных занятиях Вы получите потрясающего друга, который каждый день будет удивлять Вас своей сообразительностью.\n Ваши пожелания к интеллекту собаки:",
    "Шум": "Любой человек, решившийся завести собаку, осознаёт, что время от времени его питомец может издавать разные громкие звуки. Но и среди собак есть свои \"молчуны\", а есть любители пошуметь. Насколько Вы (и Ваши соседи) готовы к шуму? От 1 до 5.",
    "Охранные качества": "Оцените от 1 до 5, насколько Вам важны охранные качества собаки."
}

def ask_for_size(message):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    sizes = ["Маленькая", "Средняя", "Крупная", "Любая"]
    markup.add(*sizes)
    msg = bot.reply_to(message, 'Какой размер собаки вам подходит: \n маленькая (до 40см в холке);\n средняя (41-60см в холке);\n крупная (выше 60 см в холке)?', reply_markup=markup)
    bot.register_next_step_handler(msg, process_size)

def process_size(message):
    user_size = message.text.strip().lower()
    if user_size == "любая":
        user_size = None
    valid_sizes = ["маленькая", "средняя", "крупная"]
    if user_size in valid_sizes or user_size is None:
        bot.send_message(message.chat.id, f"Вы выбрали размер собаки: {user_size if user_size else 'любая'}")
        time.sleep(3)
        ask_for_characteristics(message, user_size)
    else:
        bot.send_message(message.chat.id, "Пожалуйста, выберите один или несколько из следующих вариантов: Маленькая, Средняя, Крупная или Любая")
        ask_for_size(message)

def ask_for_characteristics(message, user_size):
    bot.send_message(message.chat.id, "Дальше Вам предстоит указать пожелания к характеру собаки. ")
    time.sleep(2)
    ask_aggressiveness(message, user_size)

def ask_aggressiveness(message, user_size):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Агрессивность"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_aggressiveness, user_size)

def process_aggressiveness(message, user_size):
    try:
        user_aggressiveness = int(message.text.strip())
        if 1 <= user_aggressiveness <= 5:
            bot.send_message(message.chat.id, f"Я покажу Вам породы с уровнем агрессии не выше: {user_aggressiveness}")
            time.sleep(2)
            ask_activity(message, user_size, user_aggressiveness)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_aggressiveness(message, user_size)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_aggressiveness(message, user_size)

def ask_activity(message, user_size, user_aggressiveness):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Активность"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_activity, user_size, user_aggressiveness)

def process_activity(message, user_size, user_aggressiveness):
    try:
        user_activity = int(message.text.strip())
        if 1 <= user_activity <= 5:
            bot.send_message(message.chat.id, f"Вы указали комфортный уровень активности: {user_activity}")
            time.sleep(2)
            ask_training(message, user_size, user_aggressiveness, user_activity)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_activity(message, user_size, user_aggressiveness)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_activity(message, user_size, user_aggressiveness)


def ask_training(message, user_size, user_aggressiveness, user_activity):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Дрессировка"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_training, user_size, user_aggressiveness, user_activity)

def process_training(message, user_size, user_aggressiveness, user_activity):
    try:
        user_training = int(message.text.strip())
        if 1 <= user_training <= 5:
            bot.send_message(message.chat.id, f"В подборке не будет пород с обучаемостью ниже: {user_training}")
            time.sleep(2)
            ask_wool(message, user_size, user_aggressiveness, user_activity, user_training)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_training(message, user_size, user_aggressiveness, user_activity)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_training(message, user_size, user_aggressiveness, user_activity)



def ask_wool(message, user_size, user_aggressiveness, user_activity, user_training):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Линька"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_wool, user_size, user_aggressiveness, user_activity, user_training)

def process_wool(message, user_size, user_aggressiveness, user_activity, user_training):
    try:
        user_wool = int(message.text.strip())
        if 1 <= user_wool <= 5:
            bot.send_message(message.chat.id, f"Ваша готовность ухаживать за шерстью питомца: {user_wool}")
            time.sleep(2)
            ask_friendliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_wool(message, user_size, user_aggressiveness, user_activity, user_training)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_wool(message, user_size, user_aggressiveness, user_activity, user_training)

def ask_friendliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Дружелюбность"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_friendliness, user_size, user_aggressiveness, user_activity, user_training, user_wool)

def process_friendliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool):
    try:
        user_friendliness = int(message.text.strip())
        if 1 <= user_friendliness <= 5:
            bot.send_message(message.chat.id, f"У предложенных мной пород рейтинг дружелюбности будет не ниже: {user_friendliness}")
            time.sleep(2)
            ask_loneliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_friendliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_friendliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool)

def ask_loneliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Отношение к одиночеству"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_loneliness, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness)

def process_loneliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness):
    try:
        user_loneliness = int(message.text.strip())
        if 1 <= user_loneliness <= 5:
            bot.send_message(message.chat.id, f"Умение оставаться в одиночестве, которое Вам подходит: {user_loneliness}")
            time.sleep(2)
            ask_intellect(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_loneliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_loneliness(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness)

def ask_intellect(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Интеллект"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_intellect, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness)

def process_intellect(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness):
    try:
        user_intellect = int(message.text.strip())
        if 1 <= user_intellect <= 5:
            bot.send_message(message.chat.id, f"Требование к интеллекту питомца: {user_intellect}")
            time.sleep(2)
            ask_noise(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_intellect(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_intellect(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness)

def ask_noise(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Шум"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_noise, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect)

def process_noise(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect):
    try:
        user_noise = int(message.text.strip())
        if 1 <= user_noise <= 5:
            bot.send_message(message.chat.id, f"Подбираю породы с уровнем шума не выше: {user_noise}")
            time.sleep(2)
            ask_security(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_noise(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_noise(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect)

def ask_security(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise):
    markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
    markup.add("1", "2", "3", "4", "5")
    msg = bot.reply_to(message, bot_questions["Охранные качества"], reply_markup=markup)
    bot.register_next_step_handler(msg, process_security, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise)

def process_security(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise):
    try:
        user_security = int(message.text.strip())
        if 1 <= user_security <= 5:
            bot.send_message(message.chat.id, f"Ваши требования к охранным качествам собаки: {user_security}")
            time.sleep(2)
            filtered_df = filter_breeds(
                df,
                user_size,
                user_aggressiveness,
                user_activity,
                user_training,
                user_wool,
                user_friendliness,
                user_loneliness,
                user_intellect,
                user_noise,
                user_security
            )
            send_results(message, filtered_df)

        else:
            bot.send_message(message.chat.id, "Пожалуйста, выберите число от 1 до 5.")
            ask_security(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise)
    except ValueError:
        bot.send_message(message.chat.id, "Пожалуйста, введите число от 1 до 5.")
        ask_security(message, user_size, user_aggressiveness, user_activity, user_training, user_wool, user_friendliness, user_loneliness, user_intellect, user_noise)

def filter_breeds(df, size, aggressiveness, activity, training, wool, friendliness, loneliness, intellect, noise, security):
    filtered_df = df

    if size:
        filtered_df = filtered_df[filtered_df['Размер'].str.lower() == size]


    if aggressiveness:
        filtered_df = filtered_df[filtered_df['Агрессивность'] <= aggressiveness]

    if activity:
        filtered_df = filtered_df[filtered_df['Активность'] <= activity]

    if training:
        filtered_df = filtered_df[filtered_df['Дрессировка'] >= training]

    if wool:
        filtered_df = filtered_df[filtered_df['Линька'] <= wool]

    if friendliness:
        filtered_df = filtered_df[filtered_df['Дружелюбность'] >= friendliness]

    if loneliness:
        filtered_df = filtered_df[filtered_df['Отношение к одиночеству'] <= loneliness]

    if intellect:
        filtered_df = filtered_df[filtered_df['Интеллект'] >= intellect]

    if noise:
        filtered_df = filtered_df[filtered_df['Шум'] <= noise]

    if security:
        filtered_df = filtered_df[filtered_df['Охранные качества'] >= security]

    return filtered_df

def send_results(message, filtered_df):
    if not filtered_df.empty:
        response = "\nВот список подходящих Вам пород собак. Вы можете пройти по ссылке, познакомиться с историей породы, почитать описание, а также посмотреть фотографии. \n\n"
        for _, row in filtered_df.iterrows():
            breed_name = row['Порода']
            breed_url = next((url for name, url in porody if name == breed_name), None)
            if breed_url:
                response += f"Порода: {breed_name}, Ссылка: {breed_url}\n"
        bot.send_message(message.chat.id, response)
    else:
        bot.send_message(message.chat.id, "К сожалению, подходящих пород не найдено. Возможно, стоит немного смягчить условия поиска. Если Хотите попробовать ещё раз, нажмите:\n\n /start")

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, "Добро пожаловать! \U0001F436 \n Если не знаете, какая порода собак подойдёт Вам лучше всего, Вы обратились по адресу. Я помогу определиться. Для этого нужно ответить на несколько вопросов.")
    time.sleep(2)
    ask_for_size(message)

bot.polling(none_stop=True)